In [1]:
import os

In [2]:
import pandas as pd

In [3]:
from APRGenerator import APR_analytical_pipeline as apr

In [4]:
program="Nplate"
file_path="Nplate-PMAF-exluded.csv"

In [5]:
df=pd.read_csv(file_path,index_col=False)

In [6]:
workflow=apr(program, file_path)

In [7]:
#delete the empty rows at the end of the file
df=df.dropna(how="all")

In [8]:
df.columns

Index(['UNIT PROCEDURE', 'SAMPLING POINT', 'PARAMETER NAME',
       'PARAMETER DETAIL', 'RUN NUMBER', 'BATCH NUMBER', 'DATE OF MANUFACTURE',
       'PROCESS START', 'PARAMETER DATE', 'LOCAL ID', 'VALUE_TEXT(Reported)',
       'VALUE_NUM(Recorded Full precision)', 'UOM', 'LAL', 'UAL', 'LCL', 'UCL',
       'LRL', 'URL', 'CL', 'NEL 1', 'NEL 2', 'NEL 3', 'NEL 4', 'PPK', 'REP',
       'CONTROL_PRECISION', 'ACTION_PRECISION', 'REJECTION_PRECISION', 'SCALE',
       'SUMMARY TABLE', 'DISPOSITION_DATE', 'RESULT_TYPE', 'SOURCE_SYSTEM',
       'CTRL_VIOLATION', 'ACTION_VIOLATION', 'REJ_VIOLATION'],
      dtype='object')

In [9]:
len(df)

32

In [10]:
df=workflow.df_colname_unify(df)
# make modification on the original dataset, update the df outside the class 

In [11]:
df=workflow.specs_num_conversion(df)

In [12]:
quan_list=workflow.quanti_quali_namelists(df)[0]

In [13]:
#work only on the quantitative dataset after this 
df_quan=df[df["parameter name"].isin(quan_list)].copy()

In [14]:
df_quan=workflow.add_specside(df_quan)
df_quan=workflow.add_clside(df_quan)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [15]:
para_unit_pair_df=workflow.para_unit_combi_df(df_quan)

In [16]:
df_quan=df_quan.sort_values(by=["parameter name","unit procedure","parameter date"])

In [17]:
df_quan=workflow.spec_correction_rawset(df_quan)
#no return, make modification on df_quan

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [18]:
df_quan.columns

Index(['unit procedure', 'sampling point', 'parameter name',
       'parameter detail', 'run number', 'batch number', 'date of manufacture',
       'process start', 'parameter date', 'local id', 'value_text(reported)',
       'value_num(recorded/full precision)', 'uom', 'LAL', 'UAL', 'LCL', 'UCL',
       'lrl', 'url', 'cl', 'nel 1', 'nel 2', 'nel 3', 'nel 4', 'ppk', 'rep',
       'control_precision', 'action_precision', 'rejection_precision', 'scale',
       'summary table', 'disposition_date', 'result type', 'source_system',
       'ctrl_violation', 'action_violation', 'rej_violation', 'spec status',
       'ctrl status'],
      dtype='object')

In [19]:
OOS_list=workflow.find_oos_index(df_quan)
OOT_list=workflow.find_oot_index(df_quan)

In [20]:
OOST_list=list(set(OOS_list+OOT_list))

In [21]:
df_quan_OOS=df_quan.loc[OOS_list]
#df_quan_OOT=df_quan.loc[OOT_list]
#df_quan_no_OOST=df_quan.loc[df_quan.index.difference(OOST_list)]
df_quan_no_OOS=df_quan.loc[df_quan.index.difference(OOS_list)]
#edit this part, output dataset based on your need
df_quan_no_OOS.to_csv(program+"_quantitative_no_OOS.csv",index=False)

In [26]:
summary_df=workflow.aggre_summary_df_generator(df_quan,"",min_size=30)
summary_df_no_OOS=workflow.aggre_summary_df_generator(df_quan_no_OOS,"_no_OOS",min_size=20)

In [27]:
summary_df

,,CV,KS,LAL,N,Ppk,STD,UAL,kurtosis,mean,median,shapiro,skewness,spec status,varaince
parameter name,unit procedure,,,,,,,,,,,,,,
Product purity by CEX-HPLC - Main Peak,DP-IEDL:ADL2-ADL,0.988414,NaN,86.3,32,1.404377,0.890067,.,1.725195,90.049969,90.2415,0.012758,-1.187073,-1,0.792219


In [26]:
workflow.summary_print(summary_df, purpose="Report",input_type="")
workflow.summary_print(summary_df_no_OOS, purpose="Report",input_type="_no_OOS")

In [27]:
workflow.summary_print(summary_df, purpose="R_pearson",input_type="")
workflow.summary_print(summary_df_no_OOS, purpose="R_pearson",input_type="_no_OOS")

##### ---------------Send the output summary tables to Pearson function in R

In [52]:
summary_df_pearson=pd.read_csv(program+"_summary_stats_pearson.csv",index_col=None)
summary_df_no_OOS_pearson=pd.read_csv(program+"_summary_stats_no_OOS_pearson.csv",index_col=None)

In [53]:
#column name????
#The merged table with 4 ppks
summary_df_merged=summary_df_pearson.merge(summary_df_no_OOS_pearson,on=["unit procedure","parameter name"])

In [54]:
#can vary the threshold. Using 0.95 now bc of rounding
false_pos=summary_df_merged[(summary_df_merged["Ppk"]<0.95)&(summary_df_merged["pearson Ppk_no_OOS"]>=0.95)]
false_neg=summary_df_merged[(summary_df_merged["Ppk"]>=0.95)&(summary_df_merged["pearson Ppk_no_OOS"]<=0.95)]

In [55]:
false_pos.to_csv(program+"_false_pos.csv", index=False)
false_neg.to_csv(program+"_false_neg.csv", index=False)

In [69]:
#make directories for each parameter under false pos and false neg
#no space allowed in the folder name(check this out)
#The two lines below can not be combine to a single cell???

In [61]:
mkdir false_pos

In [65]:
mkdir false_neg

In [66]:
current_path=os.getcwd()

In [68]:
workflow.JMP_single_para_subset_generator(false_pos, df_quan_no_OOS, folder_path=current_path+"/false_pos")
workflow.JMP_single_para_subset_generator(false_neg, df_quan_no_OOS, folder_path=current_path+"/false_neg")

In [11]:
ar=pd.read_csv("Aranesp_quantitative_no_OOS.csv",index_col=False)

In [13]:
pmafs=pd.read_csv("aranesp_pmaf_summary.csv",index_col=False)

In [22]:
aranesp_pmaf_raw=pd.merge(ar,pmafs[["parameter name","unit procedure"]])

In [28]:
aranesp_pmaf_raw.to_csv("aranesp_pmaf_appx_Nov2.csv",index=False)

In [25]:
pmaf_no_OOS=aranesp.aggre_summary_df_generator(aranesp_pmaf_raw,"_no_OOS",min_size=11)

In [26]:
pmaf_no_OOS

,parameter name,unit procedure,N,mean,median,STD,variance,UAL,LAL,spec status,Ppk_no_OOS,skewness,kurtosis,CV,shapiro,KS
0,Immunoassay,PRJU:APR1-DP-ADL,30,1.032880,1.024250,0.078909,0.006227,1.2,0.8,0.0,0.705958,-0.009636,-0.425392,7.639733,0.360184,0.0
1,In vitro bioassay,PRJU:APR1-DP-ADL,30,105.770333,104.500000,6.318745,39.926541,123,78,0.0,0.908918,0.292673,-0.912300,5.974024,0.173089,0.0
2,Immunoassay,PRJU:APR1-DP-AML,30,1.048927,1.032500,0.067425,0.004546,1.2,0.8,0.0,0.746873,0.314889,-0.575502,6.427982,0.470639,0.0
3,Immunoassay,PRJU:APR14-DP-ADL,104,1.047020,1.050000,0.061025,0.003724,1.2,0.8,0.0,0.835614,0.049290,-0.034710,5.828434,0.835735,0.0
4,Immunoassay,PRJU:APR14-DP-AML,31,1.041948,1.033333,0.073056,0.005337,1.2,0.8,0.0,0.721140,-0.089767,-0.817743,7.011518,0.445171,0.0
5,CHO Proteins Immunoassay,PRJU:APR6-DS-AML,28,290.535714,289.000000,119.151159,14196.998677,632,.,1.0,0.955269,0.427893,0.034103,41.010848,0.489320,0.0
6,Isoelectric Focusing: Isoform 21 + 22,PRJU:APR6-DS-AML,29,74.458345,74.447000,4.714579,22.227255,91,54,0.0,1.169539,0.104075,-0.470185,6.331834,0.929388,0.0
7,Isoelectric Focusing: Isoform 22,PRJU:APR6-DS-AML,29,33.626310,32.748000,4.306350,18.544649,52,24,0.0,0.745125,0.205820,-0.723629,12.806489,0.629205,0.0
8,Isoelectric Focusing: Isoforms 19 + 20,PRJU:APR6-DS-AML,29,25.541845,25.553000,4.714674,22.228150,43,.,1.0,1.234313,-0.104052,-0.469961,18.458627,0.929531,0.0
